In [ ]:
import wallaroo

In [2]:
wl = wallaroo.Client()

Please log into the following URL in a web browser:

	https://dish.keycloak.wallaroo.community/auth/realms/master/device?user_code=PNDV-BYOU

Login successful!


In [3]:
ws = wl.list_workspaces()
wl.list_workspaces()

Name,Created At,Users,Models,Pipelines
praveen.mada@dish.com - Default Workspace,2022-11-30 18:42:29,['praveen.mada@dish.com'],0,0
vinayak.sharma@dish.com - Default Workspace,2022-11-30 18:44:37,"['vinayak.sharma@dish.com', 'ruyi.yang@dish.com', 'madhu.bandi@dish.com', 'evgeniya.dontsova@dish.com', 'hamza.khokhar@dish.com', 'david.cherney@dish.com', 'aakash.agrawal@dish.com', 'praveen.mada@dish.com']",1,0
dish-pov-test,2022-11-30 18:51:08,['praveen.mada@dish.com'],1,1
dish-pd-eks-ad,2022-12-02 16:49:45,"['praveen.mada@dish.com', 'evgeniya.dontsova@dish.com', 'david.cherney@dish.com', 'aakash.agrawal@dish.com', 'madhu.bandi@dish.com', 'hamza.khokhar@dish.com', 'ruyi.yang@dish.com', 'vinayak.sharma@dish.com', 'bob@wallaroo.ai']",1,1


# only run first time

In [9]:
#wrk = wl.create_workspace('test')

In [4]:
ws = wl.list_workspaces()
for w in ws:
    print(w.name())
    if w.name() == 'dish-pd-eks-ad':
        wl.set_current_workspace(w)

praveen.mada@dish.com - Default Workspace
vinayak.sharma@dish.com - Default Workspace
dish-pov-test
dish-pd-eks-ad


In [6]:
wl.get_current_workspace()

{'name': 'dish-pd-eks-ad', 'id': 65, 'archived': False, 'created_by': '7d1b6e97-94fd-4c2f-b27d-86b049936c4f', 'created_at': '2022-12-02T16:49:45.408768+00:00', 'models': [{'name': 'autoencoder-ad', 'version': '8fb8894c-9628-4a69-8e15-e7a481b89e6b', 'file_name': 'autoencoder-ad.onnx', 'image_path': None, 'last_update_time': datetime.datetime(2022, 12, 15, 18, 10, 1, 921546, tzinfo=tzutc())}], 'pipelines': [{'name': 'autoencoder-ad', 'create_time': datetime.datetime(2022, 12, 15, 18, 10, 3, 772340, tzinfo=tzutc()), 'definition': '[]'}]}

# Upload and run our model

For this example we will be using an open source model that uses an Aloha CNN LSTM model for classifiying Domain names.

## Config 
before deploying an inference engine we will set the configuration of the engine.
To do this we will use the wallaroo DeploymentConfigBuilder() and fill in the options listed below to determine what the properties of our inference engine will be

note: this will not start the process of building anything in the kubernetes cluster yet. we are just setting the deployment configuration we will want to use later.
- replica_count - 1 => when deployed this will have a single inference engine
- cpus - 14 => each inference engine will have 14 cpus
- memory - 50Gi => each inference engine will have 50 Gb of memory

# Recommedations
for this we are going to create following deployment_configration: 
 - deployment_config
 -- this config will use a single replica with 4 cpus and 8 Gb of memory and will be used for our deployment later



In [7]:
deployment_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(4).memory("4Gi").build()

# Upload model

In [8]:
model = wl.upload_model('node-autoencoder-ad', "./node_autoencoder_ad_v0.0.2_training_2022_9_29.onnx")

# Deploy a model
Now that we have a model that we want to use we will create a deployment for it. 

We will tell the deployment we are using a tensorflow model and give the deployment name and the configuration we want for the deployment

# Example

In [10]:
p = wl.build_pipeline('node-autoencoder-ad')
p = p.add_model_step(model)
pipeline = p.deploy(deployment_config = deployment_config)

 ok


# Successful deployment
now that we have a deployment running we start inferencing
(if you are still running out of resources check to make sure all other deployments have been taken down)

## infer 1 row
to test we will infer a single row of data and see the results using the infer_from_file tool
we will pass a single encoded url into the inference engine and print the results back out

In [19]:
pipeline.infer_from_file("data.txt")

[InferenceResult({'check_failures': [],
  'elapsed': 832764,
  'model_name': 'node-autoencoder-ad',
  'model_version': '3c780487-07dd-4d1a-85e9-e3ff251b963a',
  'original_data': {'tensor': [[[0.03330152, 0.01778775, 0.00088327],
                                [0.02281776, 0.01573625, 0.00212125],
                                [0.03194405, 0.01805324, 0.0013521],
                                [0.03096518, 0.01414332, 0.0016135],
                                [0.0232703, 0.01247798, 0.00180119],
                                [0.02491174, 0.01226076, 0.0016748],
                                [0.02710739, 0.01356407, 0.00175932],
                                [0.02363743, 0.01711196, 0.00155607],
                                [0.03305495, 0.01221249, 0.00190871],
                                [0.01561677, 0.01626723, 0.00138421],
                                [0.02632048, 0.01479497, 0.00168263],
                                [0.02081036, 0.01414332, 0.00155844],
     

## Run larger batch
Now we will run a larger batch. We have prepared a 1,000 inference file and a 25,000 inference file
once the results are finished they will be placed in a file titled curl_response.txt 


In [ ]:
pipeline._deployment._url()

In [ ]:
!curl -X POST {pipeline._deployment._url()} -H "Content-Type:application/json" --data @test_data10.json

In [ ]:
model2 = wl.upload_model('ccfraud-xgb', './models/xgboost_ccfraud.onnx')
pipeline.replace_with_model_step(index=0, model=model2).deploy()

## Undeploy model
This will take down our inference engine. and free up the resources in kubernetes
- Note that if the pipeline variable is unchanged deployment.deploy() will restart the inference engine in the same configuration

In [ ]:
pipeline.undeploy()